<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [ ]:
def func(x, a, b, c):
    return a * np.exp(-b * x) + c

Define the data to be fit with some noise:


In [ ]:
xdata = np.linspace(0, 4, 50)
y = func(xdata, 2.5, 1.3, 0.5)
rng = np.random.default_rng()
y_noise = 0.2 * rng.normal(size=xdata.size)
ydata = y + y_noise
plt.plot(xdata, ydata, 'b-', label='data')

Fit for the parameters a, b, c of the function `func`:


In [ ]:
popt, pcov = curve_fit(func, xdata, ydata)
popt

array([2.56274217, 1.37268521, 0.47427475])

In [ ]:
plt.plot(xdata, func(xdata, *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

Constrain the optimization to the region of ``0 <= a <= 3``,
``0 <= b <= 1`` and ``0 <= c <= 0.5``:


In [ ]:
popt, pcov = curve_fit(func, xdata, ydata, bounds=(0, [3., 1., 0.5]))
popt

array([2.43736712, 1.        , 0.34463856])

In [ ]:
plt.plot(xdata, func(xdata, *popt), 'g--',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

In [ ]:
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

For reliable results, the model `func` should not be overparametrized;
redundant parameters can cause unreliable covariance matrices and, in some
cases, poorer quality fits. As a quick check of whether the model may be
overparameterized, calculate the condition number of the covariance matrix:


In [ ]:
np.linalg.cond(pcov)

34.571092161547405  # may vary

The value is small, so it does not raise much concern. If, however, we were
to add a fourth parameter ``d`` to `func` with the same effect as ``a``:


In [ ]:
def func2(x, a, b, c, d):
    return a * d * np.exp(-b * x) + c  # a and d are redundant
popt, pcov = curve_fit(func2, xdata, ydata)
np.linalg.cond(pcov)

1.13250718925596e+32  # may vary

Such a large value is cause for concern. The diagonal elements of the
covariance matrix, which is related to uncertainty of the fit, gives more
information:


In [ ]:
np.diag(pcov)

array([1.48814742e+29, 3.78596560e-02, 5.39253738e-03, 2.76417220e+28])  # may vary

Note that the first and last terms are much larger than the other elements,
suggesting that the optimal values of these parameters are ambiguous and
that only one of these parameters is needed in the model.

If the optimal parameters of `f` differ by multiple orders of magnitude, the
resulting fit can be inaccurate. Sometimes, `curve_fit` can fail to find any
results:


In [ ]:
ydata = func(xdata, 500000, 0.01, 15)
try:
    popt, pcov = curve_fit(func, xdata, ydata, method = 'trf')
except RuntimeError as e:
    print(e)

Optimal parameters not found: The maximum number of function evaluations is
exceeded.

If parameter scale is roughly known beforehand, it can be defined in
`x_scale` argument:


In [ ]:
popt, pcov = curve_fit(func, xdata, ydata, method = 'trf',
                       x_scale = [1000, 1, 1])
popt

array([5.00000000e+05, 1.00000000e-02, 1.49999999e+01])